In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
%matplotlib inline

np.random.seed(0)

In [2]:
class NormalLR:
    def __init__(self, τ=0):
        self.τ = τ
    
    def fit(self, X, y):
        self.weights = np.linalg.inv(X.T.dot(X) + self.τ * np.eye(X.shape[1])).dot(X.T).dot(y[:,np.newaxis])
    
    def predict(self, X):
        return X.dot(self.weights).ravel()

In [3]:
class GradientLR(NormalLR):
    def __init__(self, alpha=0.1, τ=0):
        if alpha <= 0:
            raise ValueError("alpha should be positive")
        self.α = alpha
        self.τ = τ
        self.threshold = alpha / 100
        
    def fit(self, X, y):
        l = len(y)
        w = np.random.rand(X.shape[1], 1) / l - 1 / (2 * l)
        
        it = 5
        
        while True:
            w_prev = w.copy()
            w = w - self.α * (np.sum((X.dot(w) - y[:, np.newaxis]) * X, axis=0)[:, np.newaxis] / l
                              + self.τ * np.linalg.norm(w))
            if np.abs(w - w_prev).sum() < self.threshold:
                break
                
        self.weights = w

In [4]:
def mse(y_true, y_pred):
    return np.square(y_true - y_pred).mean()

In [5]:
def sample(size, weights):
    X = np.ones((size, 2))
    X[:, 1] = np.random.gamma(4., 2., size)
    y = X.dot(np.asarray(weights))
    y += np.random.normal(0, 1, size)
    return X[:, 1:], y

In [11]:
for size in [64, 128, 256, 512, 1024, 2048]:
    X, y_true = sample(size, weights=[24., 42.])
    X = X / (X.max(axis=0) - X.min(axis=0))
    err = []
    for model in [NormalLR, GradientLR]:
        lr = model()
        lr.fit(X, y_true)
        y_pred = lr.predict(X)
        err.append(mse(y_true, y_pred))
        
    print(err)

# plt.scatter(X, y_true)
# plt.plot(X, lr.predict(X), color='r')
# plt.show()

[121.97976119075041, 121.98018805693238]
[123.25128491762437, 123.25178165550771]
[114.58511495033076, 114.58554456699545]
[112.11069149257645, 112.11169218375906]
[115.3386654028703, 115.33952189277605]
[114.4274312334631, 114.42844394290509]


In [7]:
data = np.genfromtxt('boston.csv', delimiter=',')

X = data[:,:-1]
y = data[:,-1]

# X = (X - X.mean(axis=0)) / X.std(axis=0) # scaling
# X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X = X / (X.max(axis=0) - X.min(axis=0))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
for model in [NormalLR]:
    for τ in [0, 0.01, 0.1, 0.5, 1, 5]:
        lr = model(τ=τ)
        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        print("model: {m}, τ: {t}, error: {e}".format(m=model.__name__, t=τ, e=mse(y_test, y_pred)))
        print(lr.weights.T[0])
        print()

model: NormalLR, τ: 0, error: 36115317.53130742
[-10577.25840952   4559.14621681   -622.27368153   2486.45168579
    -84.28814287  31255.65021846  -1130.28442777  -9269.33644611
   2471.28712702  -4280.38362407  -4741.83708227   5639.65882566
 -13333.02671363]

model: NormalLR, τ: 0.01, error: 36116471.7818533
[-10519.87710018   4555.66426936   -631.52355712   2490.14797943
    -65.51127196  31219.00963177  -1123.25827563  -9236.75873745
   2454.18564353  -4269.44953267  -4736.50452986   5640.32195437
 -13345.27667281]

model: NormalLR, τ: 0.1, error: 36134383.55763875
[-10033.02117813   4526.42634973   -712.54713296   2522.23744682
     97.22712387  30897.55081813  -1062.89256236  -8953.48850646
   2307.19879543  -4175.0356719   -4688.99248516   5646.83336633
 -13446.07701226]

model: NormalLR, τ: 0.5, error: 36328836.02138009
[ -8365.41124526   4432.24622756  -1029.66599605   2643.70436852
    706.09506605  29622.43436255   -845.33169358  -7872.98843236
   1773.32198811  -3826.419054

In [10]:
lr = GradientLR()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("model: {m}, τ: {t}, error: {e}".format(m=model.__name__, t=τ, e=mse(y_test, y_pred)))
print(lr.weights.T[0])
print()

model: NormalLR, τ: 5, error: 36115181.32439986
[-10576.08185308   4559.07834107   -622.26945853   2486.47885806
    -84.20614854  31255.54505389  -1130.22319253  -9269.13526355
   2471.06930807  -4280.30025671  -4741.83161158   5639.64233762
 -13333.28491732]

